In [1]:
#import General Libraries
import re, string
import pandas as pd
from keybert import KeyBERT

In [2]:
#Reading Dataset
df=pd.read_csv("Womens Clothing E-Commerce Reviews.csv")

In [3]:
#Data Preparation
df.describe()

,Unnamed: 0,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000
mean,11742.500000,918.118709,43.198544,4.196032,0.822362,2.535936
std,6779.968547,203.298980,12.279544,1.110031,0.382216,5.702202
min,0.000000,0.000000,18.000000,1.000000,0.000000,0.000000
25%,5871.250000,861.000000,34.000000,4.000000,1.000000,0.000000
50%,11742.500000,936.000000,41.000000,5.000000,1.000000,1.000000
75%,17613.750000,1078.000000,52.000000,5.000000,1.000000,3.000000
max,23485.000000,1205.000000,99.000000,5.000000,1.000000,122.000000


In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df.dropna(subset=['Review Text'], inplace=True) #Removing rows where Review Text is null

In [6]:
df.Title = df.Title.fillna('') #Fill NA with blank in Title Column

In [7]:
df['Full text']=df['Title']+' '+df['Review Text'] #Concatenating Title and Review Text for analysis

In [8]:
df.head(10)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,Full text
0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,Absolutely wonderful - silky and sexy and com...
1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,Love this dress! it's sooo pretty. i happen...
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,Some major design flaws I had such high hopes ...
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"My favorite buy! I love, love, love this jumps..."
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,Flattering shirt This shirt is very flattering...
5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,Not for the very petite I love tracy reese dre...
6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,Cagrcoal shimmer fun I aded this in my basket ...
7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits,"Shimmer, surprisingly goes with lots I ordered..."
8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses,Flattering I love this dress. i usually get an...
9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses,"Such a fun dress! I'm 5""5' and 125 lbs. i orde..."


In [15]:
df=df[df['Clothing ID']==1078]

In [17]:
print(df.count())

Clothing ID                987
Age                        987
Title                      987
Review Text                987
Rating                     987
Recommended IND            987
Positive Feedback Count    987
Division Name              987
Department Name            987
Class Name                 987
Full text                  987
dtype: int64


In [18]:
#Loading Keybert Model
kw_model = KeyBERT(model='all-mpnet-base-v2')

In [19]:
def remove_similar(textlist):
    """
    Remove one Occucerence of Similar Keywords from List
    """
    result=[]
    for i in range(len(textlist)):
        present=False
        for j in range(len(textlist)):
            if i!=j and textlist[i] in textlist[j] and present==False:
                present=True
        if present==False:
            result.append(textlist[i])
    return result

In [20]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [21]:
def extractKeyword(text):
    """Method takes text as input and returns keyword list using Key Bert pretrained model"""
    keywords_list=[]
    try:
        return remove_similar(list(dict(kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 3), stop_words=get_stopwords_list("stopwords.txt"), highlight=False,top_n=10, use_maxsum=True)).keys()))
    except:
        return remove_similar(list(dict(kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 2), stop_words=get_stopwords_list("stopwords.txt"), highlight=False,top_n=10)).keys()))

In [22]:
#Getting Keywords list using extractkeyword method
df['keywords'] = df['Full text'].apply(lambda x: remove_similar(list(dict(kw_model.extract_keywords(x, keyphrase_ngram_range=(2, 2), stop_words=get_stopwords_list("stopwords.txt"), highlight=False,top_n=10)).keys())))

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/ec2-user/anaconda3/

In [23]:
print(df.head(5))

     Clothing ID  Age                   Title  \
69          1078   56    Great summer fabric!   
90          1078   51  Sweet flattering dress   
117         1078   32                           
467         1078   61    Great sweater dress!   
470         1078   33         Cute, but cheap   

                                           Review Text  Rating  \
69   I really wanted this to work. alas, it had a s...       3   
90   I love cute summer dresses and this one, espec...       4   
117  This is the perfect summer dress. it can be dr...       5   
467  Nice fit and flare style, not clingy at all. i...       5   
470  When i first opened this dress and tried it on...       3   

     Recommended IND  Positive Feedback Count   Division Name Department Name  \
69                 0                        1  General Petite         Dresses   
90                 1                        0  General Petite         Dresses   
117                1                        2  General Petite    

In [24]:
 df['keywords'].to_list()

[['summer fabric',
  'shoulders fabric',
  'fit breast',
  'fabric beautiful',
  'fit straps',
  'fabric wanted',
  'straps stay',
  'breast worked',
  'great summer',
  'strange fit'],
 ['flattering dress',
  'perfect dress',
  'summer dresses',
  'dress love',
  'dress wear',
  'dresses especially',
  'bust size',
  'curvy 38',
  'design flattering',
  'little curvy'],
 ['summer dress',
  'dressed down',
  'dress dressed',
  'down size',
  'size tie',
  'smaller bust',
  'perfect summer',
  'true size',
  'size smaller',
  'adjusted lovely'],
 ['sweater dress',
  'great sweater',
  'color petite',
  'dress nice',
  'style clingy',
  'tights boots',
  'wear tights',
  'clingy grey',
  'fit flare',
  'perfect wear'],
 ['dress hem',
  'wash dress',
  'dress tried',
  'problem hem',
  'opened dress',
  'hem rolling',
  'cute cheap',
  'ironing well',
  'hem isn',
  'solved ironing'],
 ['dress compliments',
  'dress fitted',
  'sweater dress',
  'look dress',
  'fitted skirt',
  'mentione

In [25]:
#Fetching list of keywords for clothing ID 1078
from functools import reduce
single_list = list(map(lambda x: x.lower(),reduce(lambda x,y: x+y, df['keywords'].tolist())))
print(single_list)

['summer fabric', 'shoulders fabric', 'fit breast', 'fabric beautiful', 'fit straps', 'fabric wanted', 'straps stay', 'breast worked', 'great summer', 'strange fit', 'flattering dress', 'perfect dress', 'summer dresses', 'dress love', 'dress wear', 'dresses especially', 'bust size', 'curvy 38', 'design flattering', 'little curvy', 'summer dress', 'dressed down', 'dress dressed', 'down size', 'size tie', 'smaller bust', 'perfect summer', 'true size', 'size smaller', 'adjusted lovely', 'sweater dress', 'great sweater', 'color petite', 'dress nice', 'style clingy', 'tights boots', 'wear tights', 'clingy grey', 'fit flare', 'perfect wear', 'dress hem', 'wash dress', 'dress tried', 'problem hem', 'opened dress', 'hem rolling', 'cute cheap', 'ironing well', 'hem isn', 'solved ironing', 'dress compliments', 'dress fitted', 'sweater dress', 'look dress', 'fitted skirt', 'mentioned dress', 'love sweater', 'dress dress', 'dress short', 'navy size', 'fun dress', 'dress wear', 'easy dress', 'extre

In [26]:
#Fetching unique keyword from list of keywords for clothing ID 1078 and creating dictionary of keywords with their frequency
import numpy as np

res = list(zip(*np.unique(single_list, return_counts=True)))
resDict={}
for j in res:
    resDict.update({j[0]:j[1]})
print(resDict)

{'00 petite': 1, '0p 102': 1, '0p am': 1, '0p small': 1, '10 125lbs': 1, '10 135': 1, '10 dress': 2, '10 dresses': 2, '10 fit': 3, '10 fitted': 1, '10 lining': 1, '10 looked': 1, '10 medium': 2, '10 petite': 1, '10 stomach': 1, '103 lbs': 1, '105 lbs': 2, '107 lbs': 1, '107lbs 32c': 1, '11 120': 1, '110lb pear': 1, '110lb xs': 1, '115 thank': 1, '115lbs color': 1, '115lbs fit': 1, '115lbs went': 1, '116ibs dress': 1, '12 12': 1, '12 dresses': 2, '12 fit': 1, '12 pants': 1, '120 34': 1, '120 4p': 1, '120 usual': 1, '120lbs curvy': 1, '125 lbs': 2, '125 petite': 1, '125 size': 1, '125lb fits': 1, '125lb took': 1, '125lbs large': 1, '126 pounds': 1, '128 orange': 1, '128lb ps': 1, '130 32dd': 1, '130 lbs': 2, '130 pounds': 1, '130lbs 34c': 1, '130lbs ordered': 1, '130pounds dress': 1, '134 lbs': 1, '135 bought': 1, '135 lbs': 1, '135 little': 1, '135 pounds': 1, '135 small': 1, '135lbs 34b': 1, '135lbs dress': 1, '135lbs small': 1, '138 pounds': 1, '138 sale': 1, '138lbs fits': 1, '14 12'

In [27]:
#sorting Dictionary to get most frequent keywords on top
from operator import itemgetter
sorted(resDict.items(), key=itemgetter(1), reverse=True)

[('love dress', 151),
 ('dress dress', 50),
 ('beautiful dress', 49),
 ('great dress', 48),
 ('dress love', 39),
 ('summer dress', 35),
 ('dress fit', 31),
 ('true size', 31),
 ('cute dress', 29),
 ('dress perfect', 29),
 ('dress comfortable', 28),
 ('dress looks', 28),
 ('bought dress', 27),
 ('dress beautiful', 27),
 ('dress bought', 26),
 ('dress great', 24),
 ('size down', 24),
 ('wear dress', 23),
 ('dress runs', 21),
 ('dress fits', 20),
 ('pretty dress', 20),
 ('dress just', 19),
 ('dress wear', 19),
 ('perfect dress', 18),
 ('dress am', 17),
 ('dress cute', 16),
 ('fit dress', 16),
 ('swing dress', 16),
 ('dress super', 15),
 ('ordered dress', 15),
 ('runs large', 15),
 ('runs small', 15),
 ('dress purchased', 14),
 ('dressed down', 14),
 ('retailer dresses', 14),
 ('wear size', 14),
 ('comfortable dress', 13),
 ('dress adorable', 13),
 ('dress down', 13),
 ('dress nice', 13),
 ('dress store', 13),
 ('fit perfect', 13),
 ('gorgeous dress', 13),
 ('loved dress', 13),
 ('saw dres

In [28]:
#writing Dataframe to Excel
df.to_excel("KeyBert_Keywords_Output.xlsx")

In [30]:
text="Initially I was sceptical about going for Iphone 13 mini as there were lot of bad reviews about the battery life of 12 mini. But this phone is a beast in terms of battery life and performance.I was fed up and bored using same big screen phones for years. But iphone 13 mini is perfect for my liking..Its a complete package."
print(remove_similar(list(dict(kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 3), stop_words=get_stopwords_list("stopwords.txt"), highlight=False,top_n=10, use_maxsum=True)).keys())))

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


['mini bad', 'big screen phones', 'terms battery life', 'reviews battery', 'life 12 mini', 'battery life performance', 'sceptical 13 mini', 'phones years 13', '13 mini perfect', 'mini phone beast']


In [31]:
text="Review after using it for a week. Switched to Apple because of the ever growing size of android devices. Couldn’t make a better choice. Very compact and easy to use. Battery life is excellent. Lasts entire day with moderate to heavy usage similar to my last phone. Screen size is good enough for watching videos because of zero bezels. Display quality is awesome. It took no time to get used to after switching from Oneplus seven. Excellent camera as expected from Apple. It’s the best smartphone of 2021."
print(remove_similar(list(dict(kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 3), stop_words=get_stopwords_list("stopwords.txt"), highlight=False,top_n=10, use_maxsum=True)).keys())))

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


['display quality awesome', 'phone screen size', 'switching oneplus', 'moderate heavy usage', 'last phone', 'apple best', 'android devices', 'battery life', 'excellent camera expected', 'best smartphone 2021']
